In [7]:
from pydantic import BaseModel, Field, EmailStr
from typing import List, Literal, Optional
import openai
from dotenv import load_dotenv
from datetime import date
import instructor
import os
import sys

try: 
    dotenv_loaded = load_dotenv()
    if not dotenv_loaded:
        print("Warning: no .env file found or loaded.")
except Exception as e:
    print(f"Error loading .env file: {e}")
    sys.exit(1)
    
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    print("Error: OPENAI_API_KEY not found in environment variables")
    print("Please create a .env file with: OPENAI_API_KEY=your_api_key_here")
    sys.exit(1)
    
try: 
    client = openai.OpenAI(api_key=api_key)
    print("OpenAI client successfully initialized.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    sys.exit(1)


OpenAI client successfully initialized.


In [8]:
# Pydantic data models 

class UserInput(BaseModel):
    name: str
    email: str
    query: str
    order_id: Optional[int] = Field(
        None, 
        description="5 digit (not zero) Order ID if available",
        lower_bound = 10000,
        upper_bound = 99999,
    )
    purchase_date: Optional[date] = None
    

class UserQuery(UserInput):
    priority: str = Field(
        ..., description="Priority level: low, medium, high",
    )
    category: Literal[
        'refund_request',
        'information_request',
        'other'
    ]
    is_compliant: bool = Field(
        ..., description="Whether the query is compliant or not",
    )
    tags: List[str] = Field(
        description="List of tags associated with the query",
    )

In [12]:
user_input_json = """
{
    "name": "John Doe",
    "email": "john.doe@example.com",
    "query": "I need assistance retrieving my order invoice",
    "order_id": 12345,
    "purchase_date": "2025-09-03"
}
"""

validated_user_input = UserInput.model_validate_json(user_input_json)
print(validated_user_input)

name='John Doe' email='john.doe@example.com' query='I need assistance retrieving my order invoice' order_id=12345 purchase_date=datetime.date(2025, 9, 3)
